# RL treniravimas balansavimui su Cartpole

Šis užrašų knygelė yra dalis [AI pradedantiesiems mokymo programos](http://aka.ms/ai-beginners). Ji buvo įkvėpta [oficialaus PyTorch mokymo vadovo](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) ir [šios Cartpole PyTorch implementacijos](https://github.com/yc930401/Actor-Critic-pytorch).

Šiame pavyzdyje naudosime RL, kad išmokytume modelį balansuoti stulpą ant vežimėlio, kuris gali judėti į kairę ir dešinę horizontaliai. Naudosime [OpenAI Gym](https://www.gymlibrary.ml/) aplinką, kad simuliuotume stulpą.

> **Pastaba**: Šio pamokos kodo galite vykdyti lokaliai (pvz., naudojant Visual Studio Code), tokiu atveju simuliacija atsidarys naujame lange. Vykdant kodą internete, gali reikėti atlikti tam tikrus kodo pakeitimus, kaip aprašyta [čia](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Pradėsime nuo to, kad įsitikinsime, jog Gym yra įdiegtas:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Dabar sukurkime CartPole aplinką ir pažiūrėkime, kaip su ja dirbti. Aplinka turi šias savybes:

* **Veiksmų erdvė** – tai galimų veiksmų rinkinys, kuriuos galime atlikti kiekviename simuliacijos žingsnyje  
* **Stebėjimų erdvė** – tai stebėjimų, kuriuos galime atlikti, erdvė  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Pažiūrėkime, kaip veikia simuliacija. Toliau pateikta ciklo struktūra vykdo simuliaciją tol, kol `env.step` negrąžina užbaigimo ženklo `done`. Atsitiktinai pasirinksime veiksmus naudodami `env.action_space.sample()`, o tai reiškia, kad eksperimentas greičiausiai labai greitai nepavyks (CartPole aplinka baigiasi, kai CartPole greitis, pozicija ar kampas viršija tam tikras ribas).

> Simuliacija atsidarys naujame lange. Galite paleisti kodą kelis kartus ir stebėti, kaip jis elgiasi.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Galite pastebėti, kad stebėjimai susideda iš 4 skaičių. Jie yra:
- Vežimėlio pozicija
- Vežimėlio greitis
- Stulpo kampas
- Stulpo sukimosi greitis

`rew` yra atlygis, kurį gauname kiekviename žingsnyje. Galite pastebėti, kad CartPole aplinkoje už kiekvieną simuliacijos žingsnį gaunate 1 tašką, o tikslas yra maksimaliai padidinti bendrą atlygį, t. y. laiką, per kurį CartPole sugeba išlaikyti pusiausvyrą nenukritęs.

Stiprinamojo mokymosi metu mūsų tikslas yra išmokyti **politiką** $\pi$, kuri kiekvienai būsenai $s$ nurodys, kokį veiksmą $a$ atlikti, iš esmės $a = \pi(s)$.

Jei norite probabilistinio sprendimo, galite galvoti apie politiką kaip apie grąžinančią tikimybių rinkinį kiekvienam veiksmui, t. y. $\pi(a|s)$ reikštų tikimybę, kad turėtume atlikti veiksmą $a$ būsenos $s$ metu.

## Politikos gradientų metodas

Paprasčiausiame RL algoritme, vadinamame **Politikos gradientu**, mes treniruosime neuroninį tinklą, kad jis numatytų kitą veiksmą.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Mes treniruosime tinklą vykdydami daugybę eksperimentų ir atnaujindami tinklą po kiekvieno vykdymo. Apibrėžkime funkciją, kuri vykdys eksperimentą ir grąžins rezultatus (vadinamąjį **pėdsaką**) - visas būsenas, veiksmus (ir jų rekomenduojamas tikimybes) bei atlygius:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Galite paleisti vieną epizodą su netreniruotu tinklu ir pastebėti, kad bendras atlygis (dar žinomas kaip epizodo trukmė) yra labai mažas:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Viena iš sudėtingų politikos gradientų algoritmo aspektų yra naudoti **diskontuotus apdovanojimus**. Idėja yra ta, kad mes apskaičiuojame bendrų apdovanojimų vektorių kiekviename žaidimo žingsnyje, o šio proceso metu ankstyvus apdovanojimus diskontuojame naudodami tam tikrą koeficientą $gamma$. Taip pat normalizuojame gautą vektorių, nes jį naudosime kaip svorį, kuris paveiks mūsų mokymąsi:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Dabar pradėkime mokymą! Mes vykdysime 300 epizodų, o kiekviename epizode atliksime šiuos veiksmus:

1. Vykdysime eksperimentą ir surinksime seką.
2. Apskaičiuosime skirtumą (`gradients`) tarp atliktų veiksmų ir numatytų tikimybių. Kuo mažesnis skirtumas, tuo labiau esame įsitikinę, kad pasirinkome teisingą veiksmą.
3. Apskaičiuosime diskontuotus apdovanojimus ir padauginsime gradientus iš diskontuotų apdovanojimų – tai užtikrins, kad žingsniai su didesniais apdovanojimais turės didesnį poveikį galutiniam rezultatui nei tie, kurie gavo mažesnius apdovanojimus.
4. Tikėtini tiksliniai veiksmai mūsų neuroniniam tinklui bus iš dalies paimti iš numatytų tikimybių eksperimentų metu, o iš dalies iš apskaičiuotų gradientų. Naudosime `alpha` parametrą, kad nustatytume, kiek gradientai ir apdovanojimai bus įtraukti – tai vadinama *mokymosi greičiu* stiprinimo algoritmo.
5. Galiausiai, treniruosime tinklą pagal būsenas ir tikėtinus veiksmus, ir kartosime procesą.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Dabar paleiskime epizodą su atvaizdavimu, kad pamatytume rezultatą:


In [ ]:
_ = run_episode(render=True)

Tikimės, kad dabar galite matyti, jog stulpas gali gana gerai išlaikyti pusiausvyrą!

## Aktoriaus-Kritiko modelis

Aktoriaus-Kritiko modelis yra tolesnė politikos gradientų plėtra, kurioje mes kuriame neuroninį tinklą, kad išmoktume tiek politiką, tiek numatomus apdovanojimus. Tinklas turės du išėjimus (arba galite tai laikyti dviem atskirais tinklais):
* **Aktorius** rekomenduos veiksmą, kurį reikia atlikti, pateikdamas mums būsenos tikimybių pasiskirstymą, kaip ir politikos gradientų modelyje.
* **Kritikas** įvertins, kokie galėtų būti apdovanojimai už tuos veiksmus. Jis grąžina bendrą numatomą apdovanojimų sumą ateityje esant tam tikrai būsenai.

Apibrėžkime tokį modelį:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Mums reikėtų šiek tiek pakeisti mūsų `discounted_rewards` ir `run_episode` funkcijas:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Dabar vykdysime pagrindinį mokymo ciklą. Naudosime rankinį tinklo mokymo procesą, apskaičiuodami tinkamas nuostolių funkcijas ir atnaujindami tinklo parametrus:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Svarbiausia

Šioje demonstracijoje matėme du RL algoritmus: paprastą politikos gradientą ir sudėtingesnį aktoriaus-kritiko metodą. Galite pastebėti, kad šie algoritmai veikia su abstrakčiomis būsenos, veiksmo ir atlygio sąvokomis – todėl juos galima taikyti labai skirtingose aplinkose.

Pastiprinimo mokymasis leidžia mums išmokti geriausią strategiją problemos sprendimui, tiesiog stebint galutinį atlygį. Tai, kad mums nereikia pažymėtų duomenų rinkinių, leidžia daug kartų kartoti simuliacijas, siekiant optimizuoti mūsų modelius. Tačiau RL srityje vis dar yra daug iššūkių, kuriuos galite išmokti, jei nuspręsite gilintis į šią įdomią dirbtinio intelekto sritį.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
